In [1]:
!git clone https://huggingface.co/THUDM/chatglm2-6b

Cloning into 'chatglm2-6b'...
remote: Enumerating objects: 177, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 177 (delta 60), reused 10 (delta 10), pack-reused 76
Receiving objects: 100% (177/177), 1.94 MiB | 4.08 MiB/s, done.
Resolving deltas: 100% (87/87), done.
Filtering content: 100% (8/8), 11.63 GiB | 98.52 MiB/s, done.


In [2]:
!git clone https://github.com/THUDM/ChatGLM2-6B.git

Cloning into 'ChatGLM2-6B'...
remote: Enumerating objects: 277, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 277 (delta 150), reused 103 (delta 101), pack-reused 87
Receiving objects: 100% (277/277), 7.36 MiB | 17.73 MiB/s, done.
Resolving deltas: 100% (155/155), done.


In [3]:
!pip install -r /content/ChatGLM2-6B/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 9.6 MB/s eta 0:00:0

In [4]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.2 MB/s eta 0:00:00


In [5]:
from transformers import AutoTokenizer, AutoModel, AutoConfig
from peft import PeftConfig, PeftModel, LoraConfig, get_peft_model, TaskType
import torch

In [6]:
def load_lora_config(model):
    config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.1,
        target_modules=["query_key_value"]
    )
    return get_peft_model(model, config)

In [14]:
device = torch.device("cuda:0")

model_name = "/content/chatglm2-6b"
lora_dir = "/content/output"

In [12]:
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [16]:
config = PeftConfig.from_pretrained(lora_dir)
model = PeftModel.from_pretrained(model, lora_dir)

In [17]:
model = model.to(device)
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): PeftModelForCausalLM(
          (base_model): LoraModel(
            (model): ChatGLMForConditionalGeneration(
              (transformer): ChatGLMModel(
                (embedding): Embedding(
                  (word_embeddings): Embedding(65024, 4096)
                )
                (rotary_pos_emb): RotaryEmbedding()
                (encoder): GLMTransformer(
                  (layers): ModuleList(
                    (0-27): 28 x GLMBlock(
                      (input_layernorm): RMSNorm()
                      (self_attention): SelfAttention(
                        (query_key_value): Linear(
                          in_features=4096, out_features=4608, bias=True
                          (lora_dropout): ModuleDict(
                            (default): Dropout(p=0.1, inplace=False)
                          )
                          (lora_A): M

In [18]:
!nvidia-smi

Tue Sep 19 09:03:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    44W / 300W |  12788MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
response, history = model.chat(tokenizer, "5月至今上腹靠右隐痛，右背隐痛带酸，便秘，喜睡，时有腹痛，头痛，腰酸症状？", history=[])

In [20]:
response

'这些症状可能是胃肠道问题或肝胆疾病引起的。如果疼痛位于上腹部靠右，可能是由肝胆囊炎、胆囊结石或其他肝胆疾病引起的。右背隐痛带酸可能是由肝棘球辏炎、肝囊肿或其他肝病引起的。便秘、喜睡、时有腹痛可能是由胃肠功能紊乱、肠道感染或其他因素引起的。头痛和腰酸可能是由紧张、缺乏睡眠或其他因素引起的。\n\n如果这些症状持续了5个月以上，建议您咨询医生以获取更详细的诊断和治疗建议。医生可能会进行超声检查、 CT 扫描或其他测试来帮助确认症状的原因，并提供适当的治疗方案。此外，注意饮食健康、保持足够的睡眠和锻炼适当可以帮助改善胃肠功能和减轻疼痛。'